# Hotel Comment

In [1]:
import pandas as pd
import numpy as np

##利用pandas讀取資料

data_train = pd.read_csv('C:/Users/carter840826/01_Hoteltrain.csv')
data_test = pd.read_csv('C:/Users/carter840826/01_Hoteltest.csv')

##複製一個訓練資料

data1 = data_train.copy(deep = True)


##檢查訓練、驗證資料資訊及有無空白

print(data1.head())
print(data1.info())
print("-"*10)
print('Train columns with null values:\n', data_train.isnull().sum())
print("-"*10)
print('Test columns with null values:\n', data_test.isnull().sum())

   index  label                                             review
0      1      0  來往虹橋機場,絕對方便,938公共汽車直接門對門,上車睡覺就可以了(一個半小時)賓館服務不專...
1      2      1  來武漢出差很多次了，這次經朋友介紹，住在海怡錦江大酒店。外表看來的確一般，但住進去後，感覺還...
2      3      0  來長春之前，查閱了攜程的很多酒店評分，感覺這個還算不錯的，但是入住之後感覺很失望：1、房間門...
3      4      1  來蘇州好多次了，基本上都住在這家酒店。給我的感覺非常好，房間很乾淨，設施也很全，各種用品也很...
4      5      1  依海而建，地理位置絕佳，旁邊就是星海廣場。床特別大，很舒適，房間裡有咖啡包，但有一天服務生忘...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
index     5000 non-null int64
label     5000 non-null int64
review    5000 non-null object
dtypes: int64(2), object(1)
memory usage: 117.3+ KB
None
----------
Train columns with null values:
 index     0
label     0
review    0
dtype: int64
----------
Test columns with null values:
 index     0
label     0
review    0
dtype: int64


In [2]:
import jieba

##載入中文斷詞

fp = open(r'C:/Users/carter840826/Chinese stopwords.txt',encoding='utf-8')
stopwords = fp.readlines()
stopwords = [w.replace('\n', '') for w in stopwords]

##將訓練資料集合成新的DataFarme，多添加segment之後放入處理過的句子

corpos = pd.DataFrame({
    'review': data1['review'], 
    'label': data1['label'],
    'segment': 'null'
    })

##載入jieba，利用for迴圈處理每個句子、分詞以及移除斷詞
##由於電腦速度問題，這邊無法使用posseg更精確地以詞性分詞，可能會造成之後正確率的問題

for i in range(len(corpos['review'])):
    segments = []
    segment = []
    label = corpos['label'][i]
    review = corpos['review'][i]
    segs = jieba.cut(review)
    for seg in segs:
        segs = []
        if seg not in stopwords:
            segs = segments.append(seg)
    
    corpos['segment'][i] = segments
    
##將處理好的分詞與label組成新的DataFrame，print出來看看成果
    
segmentDataFrame = pd.DataFrame({
    'segment': corpos['segment'], 
    'label': corpos['label']
    })
print(segmentDataFrame)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\CARTER~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.139 seconds.
Prefix dict has been built succesfully.
C:\Users\carter840826\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                                                segment  label
0     [往虹橋, 機場, 方便, 938, 公共, 汽車, 直接, 門對門, 上車, 睡覺, 一個...      0
1     [武漢, 出差, 這次經, 朋友, 介紹, 住, 海怡錦江, 大酒店, 外表, 住, 後, ...      1
2     [來長, 春, 之前, 查閱, 攜程, 酒店, 評分, 感覺, 還算, 不錯, 入住, 之後...      0
3     [蘇州, 好, 住, 這家, 酒店, 給我, 感覺, 好, 房間, 乾淨, 設施, 全, 用...      1
4     [依海而建, 地理位置, 絕佳, 旁邊, 星海, 廣場, 床特別, 很舒適, 房間裡, 咖啡...      1
...                                                 ...    ...
4995  [攜程, 推薦, 總台, 登記, 後, 房間, 掃, 大堂, 一下, 一小, 時, 沒人, ...      0
4996  [攜程, 推薦, 星級, 酒店, 實際, 感覺, 房間裡, 招待所, 小不說, 髒, 亂, ...      0
4997  [攜程, 這次, 不錯, 訂, 行政, 房給, 免費, 升級, 號樓, 山景, 房, 酒店,...      1
4998  [攜程, 提供, 160, 元, 普通, 標間, 競爭力, 房間, 設施, 床單, 破損, ...      0
4999  [攜程, 當天, 預定, 房間, 個小時, 不到, 時間, 達, 酒店, 卻, 告知, 已經...      0

[5000 rows x 2 columns]


In [3]:
import keras

##利用keras將文字轉換為相對應的數值，MAX_NUM_WORDS代表轉換字典內最多能包含的數目

MAX_NUM_WORDS = 12500
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)

corpus_x = segmentDataFrame.segment
print(pd.DataFrame(corpus_x.iloc[:5],columns=['segment']))

##將處理好的分詞套入上面設定的tokenizer建立字典並轉換為相對應的數值
##print x_train與y_train的長度以確定資料量

tokenizer.fit_on_texts(corpus_x)
x_train = tokenizer.texts_to_sequences(corpus_x)
y_train = segmentDataFrame.label
print(len(x_train),len(y_train))

##列出每句話的轉換後字串長度，並列出前5個數值
##計算字串長度的max值以及mean值

for seq in x_train[:5]:
    print(len(seq), seq[:5], '...')
max_seq_len = max([len(seq) for seq in x_train])
mean_seq_len = sum([len(seq) for seq in x_train])/len([len(seq) for seq in x_train])
print(max_seq_len)
print(mean_seq_len)

##將字串用Zero Padding設定在同樣的字串長度=35
##不足的在前方以0補齊

MAX_SEQUENCE_LENGTH = 35
x_train = keras.preprocessing.sequence.pad_sequences(
    x_train,
    maxlen=MAX_SEQUENCE_LENGTH)

##將y_train轉換為One-hot Encoding，以便之後在決定答案時有機率值作依據

y_train = keras.utils.to_categorical(y_train)


##再確定一次轉換後的句子數值以及答案數值

print(x_train[:5])
print(y_train[:5])

Using TensorFlow backend.


                                             segment
0  [往虹橋, 機場, 方便, 938, 公共, 汽車, 直接, 門對門, 上車, 睡覺, 一個...
1  [武漢, 出差, 這次經, 朋友, 介紹, 住, 海怡錦江, 大酒店, 外表, 住, 後, ...
2  [來長, 春, 之前, 查閱, 攜程, 酒店, 評分, 感覺, 還算, 不錯, 入住, 之後...
3  [蘇州, 好, 住, 這家, 酒店, 給我, 感覺, 好, 房間, 乾淨, 設施, 全, 用...
4  [依海而建, 地理位置, 絕佳, 旁邊, 星海, 廣場, 床特別, 很舒適, 房間裡, 咖啡...
5000 5000
21 [10593, 156, 14, 10594, 2078] ...
124 [5263, 294, 10597, 80, 426] ...
26 [5266, 2420, 347, 10610, 20] ...
95 [1156, 3, 7, 56, 1] ...
39 [10614, 110, 2422, 195, 3720] ...
447
36.3958
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0 10593   156    14 10594  2078   727   251 10595  2419   405
     19   832    23     6   578    54  6881   185   176   113 10596]
 [  137     7    65     1   156   949 10608    53   268     5   156   949
   1365   918   156   169  1366   407     1 10609   580     1   580  1814
     14     2   174   119    37    24     5     1   126   655   678]
 [    0     0     0     0     0     0     0     0    

In [4]:
from sklearn.model_selection import train_test_split

##利用sklearn裡面的train_test_split切割訓練資料
##VALIDATION_RATIO=0.1代表切割資料比率為9:1

VALIDATION_RATIO = 0.1
RANDOM_STATE = 10

x_train,x_val,y_train,y_val = train_test_split(
    x_train, 
    y_train, 
    test_size=VALIDATION_RATIO, 
    random_state=RANDOM_STATE)

##確認訓練集合(Trainning Set)以及驗證集合(Validation Set)大小

print("Training Set")
print("-" * 10)
print(f"x_train : {x_train.shape}")
print(f"y_train : {y_train.shape}")
print("-" * 10)
print("Validation Set")
print("-" * 10)
print(f"x_val :   {x_val.shape}")
print(f"y_val :   {y_val.shape}")


Training Set
----------
x_train : (4500, 35)
y_train : (4500, 2)
----------
Validation Set
----------
x_val :   (500, 35)
y_val :   (500, 2)


In [5]:
from tensorflow.keras import Input
from tensorflow.keras.layers import Embedding, LSTM, concatenate, Dense
from tensorflow.keras.models import Model

##利用keras內的模組建立RNN模型，RNN為有記憶力的循環網路，會依序讀取list裡面的值以後，計算並做出回饋
##利用LSTM作為RNN模型內的細胞，LSTM的優點為可以將記憶狀態儲存
##其中NUM_CLASSES為答案種類(0,1)；NUM_EMBEDDING_DIM為一個詞向量的維度；NUM_LSTM_UNITS為LSTM輸出的向量維度

NUM_CLASSES = 2
MAX_NUM_WORDS = 12500
NUM_EMBEDDING_DIM = 256
NUM_LSTM_UNITS = 128
MAX_SEQUENCE_LENGTH = 35

##設定模型內的Input data以及Output data
##經過詞嵌入層的轉換，每則評論都變成一個詞向量的序列
##而每個詞向量的維度為256

data_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
embedding_layer = Embedding(MAX_NUM_WORDS, NUM_EMBEDDING_DIM)
data_embedded = embedding_layer(data_input)

##評論經過LSTM層後轉換為一個128維度的向量

shared_lstm = LSTM(NUM_LSTM_UNITS)
data_output = shared_lstm(data_embedded)

##利用Softmax Activation傳回兩個值，分別代表可能的機率

dense =  Dense(units=NUM_CLASSES, activation='softmax')
predictions = dense(data_output)

#建立模型

model = Model(inputs=data_input, outputs=predictions)
print(model.summary())

##定義損失函數的模型
##binary_crossentropy為損失函數內的交叉熵；accuracy可以用來了解模型正確度
##優化器選擇rmsprop

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 35)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 35, 256)           3200000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               197120    
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 3,397,378
Trainable params: 3,397,378
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
BATCH_SIZE = 128
NUM_EPOCHS = 6

##利用訓練資料擬合模型
##BATCH_SIZE為梯度下降法每更新一次參數值所跑完的資料數目；NUM_EPOCHS為RNN模型迭代次數

history = model.fit(
    x=x_train, 
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    
    validation_data=(
        x_val, 
        y_val
    ),
    
    shuffle=True
)

Train on 4500 samples, validate on 500 samples
Epoch 1/6
4500/4500 [==============================] - 9s 2ms/sample - loss: 0.4951 - accuracy: 0.7573 - val_loss: 0.3677 - val_accuracy: 0.8140
Epoch 2/6
4500/4500 [==============================] - 4s 991us/sample - loss: 0.2508 - accuracy: 0.9060 - val_loss: 0.3665 - val_accuracy: 0.8380
Epoch 3/6
4500/4500 [==============================] - 4s 980us/sample - loss: 0.1549 - accuracy: 0.9460 - val_loss: 0.4498 - val_accuracy: 0.8740
Epoch 4/6
4500/4500 [==============================] - 4s 981us/sample - loss: 0.0913 - accuracy: 0.9691 - val_loss: 0.4986 - val_accuracy: 0.8440
Epoch 5/6
4500/4500 [==============================] - 5s 1ms/sample - loss: 0.0587 - accuracy: 0.9818 - val_loss: 0.6316 - val_accuracy: 0.8480
Epoch 6/6
4500/4500 [==============================] - 5s 1ms/sample - loss: 0.0369 - accuracy: 0.9873 - val_loss: 0.7152 - val_accuracy: 0.8480


In [7]:
##利用同樣的方法輸入01_Hoteltest並轉換為模型內可用數值

corpos_test = pd.DataFrame({
    'review': data_test['review'], 
    'label': data_test['label'],
    'segment': 'null'
    })

for i in range(len(corpos_test['review'])):
    segments = []
    segment = []
    label = corpos_test['label'][i]
    review = corpos_test['review'][i]
    segs = jieba.cut(review)
    for seg in segs:
        segs = []
        if seg not in stopwords:
            segs = segments.append(seg)
    
    corpos_test['segment'][i] = segments
    
segment_test = pd.DataFrame({
    'segment': corpos_test['segment'], 
    'label': corpos_test['label']
    })


corpus_test = segment_test.segment

tokenizer.fit_on_texts(corpus_test)
x_test = tokenizer.texts_to_sequences(corpus_test)

x_test = keras.preprocessing.sequence.pad_sequences(
    x_test,
    maxlen=MAX_SEQUENCE_LENGTH)


C:\Users\carter840826\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
##利用上面建立好的模型預測測試集
##輸出預測結果

predictions = model.predict(x_test)
print(predictions[:10])

data_test['label'] = np.argmax(predictions, axis=1)

submission = data_test.loc[:, ['label']].reset_index()

submission.columns = ['index','label']
print(submission[:10])

##將預測結果存成csv檔

submission.to_csv('C:/Users/carter840826/01_Hoteltest(hsu).csv')

data_test.head()

[[3.6565495e-01 6.3434500e-01]
 [2.3280169e-01 7.6719832e-01]
 [8.2911383e-03 9.9170882e-01]
 [2.3722029e-03 9.9762779e-01]
 [4.5220327e-02 9.5477962e-01]
 [9.7592825e-01 2.4071775e-02]
 [1.3780651e-05 9.9998617e-01]
 [5.2148625e-05 9.9994791e-01]
 [9.4583929e-01 5.4160729e-02]
 [3.0432528e-01 6.9567472e-01]]
   index  label
0      0      1
1      1      1
2      2      1
3      3      1
4      4      1
5      5      0
6      6      1
7      7      1
8      8      0
9      9      1


,index,label,review
0,1,1,"""此期間預訂，入住首日酒店贈送每間房10元洗衣券一張，通過攜程預訂，入住首日每房還可獲贈歡迎..."
1,2,1,&#35828;&#23454;&#35805;，&#23545;景&#21306;酒店的硬...
2,3,1,(1)房間衛生乾淨空間大!(2)早餐美味風富菜色多!
3,4,1,（1）酒店冊子介紹說房間內提供飲用水，水壺內沒有水，給前臺提意見。前臺說飲用水就是衛生間的自...
4,5,1,*房間很不錯，服務很好，就是位置偏點，在機場到市區的路邊，打車到江北商業圈起步價。*早餐不錯。
